In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
#import os
import pickle
#import joblib
from google.cloud import storage
#import datetime

In [8]:
train_data = pd.read_csv("gs://lifesight-data-table/MLops/train_data.csv")
y_train = train_data['y_train']
X_train = train_data.drop(columns=['y_train'])

In [9]:
lr = LinearRegression(normalize=True,fit_intercept=True,n_jobs=1)
lr.fit(X_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


LinearRegression(n_jobs=1, normalize=True)

In [13]:
filename = 'model.sav'
joblib.dump(lr,open(filename,'wb'))

In [14]:
BUCKET_NAME = 'lifesight-data-table'

In [15]:
bucket = storage.Client().bucket(BUCKET_NAME)
blob = bucket.blob(str('MLops/')+filename)
blob.upload_from_filename(filename)

In [5]:
filename = 'gs://lifesight-data-table/MLops/model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'gs://lifesight-data-table/MLops/model.sav'

In [2]:
from tempfile import TemporaryFile

In [3]:
bucket_name='lifesight-data-table'
model_bucket='MLops/model.sav'

In [6]:
bucket = storage.Client().get_bucket(bucket_name)
blob = bucket.blob(model_bucket)

In [7]:
blob

<Blob: lifesight-data-table, MLops/model.sav, None>

In [10]:
with TemporaryFile() as temp_file:
    #download blob into temp file
    blob.download_to_file(temp_file)
    temp_file.seek(0)
    #load into joblib
    #model=joblib.load(temp_file)
    model = pickle.load(temp_file)

In [11]:
test_data = pd.read_csv("gs://lifesight-data-table/MLops/test_data.csv")
y_test = test_data['y_test']
X_test = test_data.drop(columns=['y_test'])

In [12]:
model.predict(X_test)

array([5.49183046, 5.70626995, 4.92876902, ..., 5.98766831, 5.62759317,
       5.73206845])

In [13]:
y_test


0       5.509203
1       5.838849
2       5.152288
3       5.748188
4       5.736397
          ...   
4318    5.587252
4319    5.537756
4320    6.060698
4321    5.793930
4322    5.653213
Name: y_test, Length: 4323, dtype: float64